In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import itertools
from skimage.transform import resize
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from skimage.feature import local_binary_pattern 
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import confusion_matrix, classification_report

In [2]:
def show_dataset(images_class, label):
    # show data for 1 class
    plt.figure(figsize=(14,5))
    k = 0
    for i in range(1,6):
        plt.subplot(1,5,i)
        try :
            plt.imshow(images_class[k][:,:,::-1])
        except :
            plt.imshow(images_class[k], cmap='gray')
        plt.title(label)
        plt.axis('off')
        plt.tight_layout()
        k += 1
    plt.show()

In [22]:
dataset_folder = 'E:\Ripki Alfinnur\Belajar_Ai\Metode LBP\Dataset1'

names = []
images = []
for folder in os.listdir(dataset_folder):
    for name in os.listdir(os.path.join(dataset_folder, folder))[:100]: # 100 images per class
        img = cv2.imread(os.path.join(dataset_folder + folder, name))
        img_grey = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
        img_resized = cv2.resize(img_grey, (100, 100))
        images.append(img_resized)
        names.append(folder)

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [20]:
labels = np.unique(names)

labels

array([], dtype=float64)

In [ ]:
for label in labels:
    
    ids = np.where(label== np.array(names))[0]
    images_class = images[ids[0] : ids[-1] + 1]
    show_dataset(images_class, label)
    plt.show()


In [ ]:
le = LabelEncoder()

le.fit(names)

print(le.classes_)

In [ ]:
name_vec = le.transform(names)

In [ ]:
print(name_vec)

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(np.array(images, dtype=np.float32),   # input data
                                                    np.array(name_vec),                   # target/output data 
                                                    test_size=0.2, 
                                                    random_state=42)

In [ ]:
print(x_train.shape, y_train.shape, x_test.shape,  y_test.shape)

In [ ]:
P = 8
R = 1
img = x_train[0]
lbp_img = local_binary_pattern(img, P=P, R=R, method="default")

In [ ]:
plt.imshow(lbp_img, cmap="gray")

In [ ]:
H = np.histogram(lbp_img.ravel(),                          
                 bins=2**P,                          
                 range=(0, 2**P),
                 density=True)[0] 

In [ ]:
print(H)

In [ ]:
plt.figure(figsize=(7,3))
plt.subplot(1,2,1)
plt.imshow(img, cmap="gray")

plt.subplot(1,2,2)
axis = np.arange(len(H))
plt.bar(axis, H)

plt.show

In [ ]:
class LBPImageClassification():
    #
    # ----- class for LBPH + Nearest Neighbours (chi 2) ------
    #
    def __init__(self):
        self.neigh = NearestNeighbors(n_neighbors=3, radius=0.4, metric=self.chi2_distance)
        self.image_histograms = []
        self.y = []
    
    def chi2_distance(self, hist1, hist2, gamma=0.5): 
        chi = gamma * np.sum(((hist1 - hist2) ** 2) / (hist1 + hist2 + 1e-7))
        return chi
    
    def find_lbp_histogram(self, image, P=8, R=1, eps=1e-7, n_window=(8,8)):
        E = []
        h, w = image.shape
        h_sz = int(np.floor(h/n_window[0]))
        w_sz = int(np.floor(w/n_window[1]))
        lbp_img = local_binary_pattern(image, P=P, R=R, method="default")
        for (x, y, C) in self.sliding_window(lbp_img, stride=(h_sz, w_sz), window=(h_sz, w_sz)):
            if C.shape[0] != h_sz or C.shape[1] != w_sz:
                continue
            H = np.histogram(C,                          
                             bins=2**P, 
                             range=(0, 2**P),
                             density=True)[0] 
            
            H = H.astype("float")
            H /= (H.sum() + eps)
            E.extend(H)
        return E
    
    def sliding_window(self, image, stride, window):
        for y in range(0, image.shape[0], stride[0]):
            for x in range(0, image.shape[1], stride[1]):
                yield (x, y, image[y:y + window[1], x:x + window[0]])
                   
    def train(self, x, y):
        self.y = y
        self.image_histograms = [self.find_lbp_histogram(img) for img in x]
        self.neigh.fit(self.image_histograms)
    
    def predict(self, x):
        hists = [self.find_lbp_histogram(img) for img in x]
        dist, idx = self.neigh.kneighbors(hists)
        pred_id = np.array([self.y[i[0]] for i in idx])
        return pred_id, dist[0]

In [ ]:
model_lbp = LBPImageClassification()

In [ ]:
model_lbp.train(x_train, y_train)

In [ ]:
y_predict = model_lbp.predict(x_test)[0]
print("The predicted Data is :")
print(y_predict)
print("The actual data is:")
print(np.array(y_test))

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(6, 6))
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()

In [ ]:
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_predict)
np.set_printoptions(precision=2)


# Plot non-normalized confusion matrix
plot_confusion_matrix(cnf_matrix, classes=labels,normalize=False,
                      title='Confusion matrix For - LBPH')

In [ ]:
print("== Classification Report for - LBP ==\n")
print(classification_report(y_test, 
                            y_predict, 
                            target_names=labels))

In [ ]:
import pickle
saved_model = '/content/drive/MyDrive/save_model/Pakcoy_Classifier.p'
pickle.dump(model_lbp,open(saved_model,'wb'))
print("Pickle is dumped successfully")

In [ ]:
saved_model = '/content/drive/MyDrive/save_model/Pakcoy_Classifier.p'
model=pickle.load(open(saved_model,'rb'))